Nama: Aisha Dian Purinra

NIM: 11210910000064

Judul: Klasifikasi Kualitas Wortel Berdasarkan Citra Digital Menggunakan Ekstraktor Fitur MobileNetV2 dan SVM Multikelas

# SETUP ENVIRONMENT

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import time
from google.colab import drive

print("Menghubungkan ke Google Drive...")
drive.mount('/content/drive')
print("Google Drive berhasil terhubung.")

Menghubungkan ke Google Drive...
Mounted at /content/drive
Google Drive berhasil terhubung.


In [ ]:
BASE_PATH = '/content/drive/MyDrive/Dataset_wortel'
TRAIN_DIR = os.path.join(BASE_PATH, 'train')
TEST_DIR = os.path.join(BASE_PATH, 'test')

# Parameter Model
IMG_SIZE = (224, 224)
NUM_CLASSES = 6
BATCH_SIZE = 32

if not os.path.exists(TRAIN_DIR) or not os.path.exists(TEST_DIR):
    print("Error: Direktori dataset tidak ditemukan!")
    print(f"Pastikan path '{BASE_PATH}' sudah benar dan berisi folder 'train' dan 'test'.")
else:
    print(f"Direktori dataset ditemukan di: {BASE_PATH}")

Direktori dataset ditemukan di: /content/drive/MyDrive/Dataset_wortel


# LOAD EKSTRAKTOR FITUR (MobileNetV2)

In [ ]:
def create_feature_extractor(num_features=1000):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    feature_output = Dense(num_features, activation=tf.nn.relu6)(x)

    model = Model(inputs=base_model.input, outputs=feature_output)
    return model

print("Membuat model MobileNetV2 untuk ekstraksi fitur...")
feature_extractor_model = create_feature_extractor(num_features=1000)
feature_extractor_model.summary()
print("Model ekstraktor fitur berhasil dibuat.")

Membuat model MobileNetV2 untuk ekstraksi fitur...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,538,984 (13.50 MB)

 Trainable params: 1,281,000 (4.89 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Model ekstraktor fitur berhasil dibuat.


# EKSTRAKSI FITUR

In [ ]:
def extract_features(directory, model):
    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    generator = datagen.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )


    num_images = len(generator.filenames)
    print(f"Mengekstrak fitur dari {num_images} gambar di {directory}...")

    features = model.predict(generator, steps=len(generator), verbose=1)
    labels = generator.classes

    filenames = generator.filenames

    return features, labels, filenames

train_features, train_labels, train_filenames = extract_features(TRAIN_DIR, feature_extractor_model)

test_features, test_labels, test_filenames = extract_features(TEST_DIR, feature_extractor_model)

print("\nEkstraksi fitur selesai.")
print(f"Dimensi fitur training: {train_features.shape}")
print(f"Dimensi label training: {train_labels.shape}")
print(f"Dimensi fitur testing: {test_features.shape}")
print(f"Dimensi label testing: {test_labels.shape}")

Found 3072 images belonging to 6 classes.
Mengekstrak fitur dari 3072 gambar di /content/drive/MyDrive/Dataset_wortel/train...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


96/96 ━━━━━━━━━━━━━━━━━━━━ 2121s 22s/step
Found 96 images belonging to 6 classes.
Mengekstrak fitur dari 96 gambar di /content/drive/MyDrive/Dataset_wortel/test...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 10s/step

Ekstraksi fitur selesai.
Dimensi fitur training: (3072, 1000)
Dimensi label training: (3072,)
Dimensi fitur testing: (96, 1000)
Dimensi label testing: (96,)


In [ ]:
# SIMPAN FITUR EKSTRAKSI KE GOOGLE DRIVE
print("\nMenyimpan hasil ekstraksi fitur...")

save_dir = '/content/drive/MyDrive/bismillah/features'
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, 'train_features.npy'), train_features)
np.save(os.path.join(save_dir, 'train_labels.npy'), train_labels)
np.save(os.path.join(save_dir, 'test_features.npy'), test_features)
np.save(os.path.join(save_dir, 'test_labels.npy'), test_labels)

print(f"Fitur dan label disimpan di folder: {save_dir}")


Menyimpan hasil ekstraksi fitur...
Fitur dan label disimpan di folder: /content/drive/MyDrive/bismillah/features


# EKSTRAKSI FITUR SELESAI